# Word2Vec con Gensim

## Instalación de librerías y carga de dataset

In [ ]:
# !pip install datasets

In [1]:
from gensim.models import Word2Vec
import pandas as pd
import re
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short, stem_text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

-------------------------

In [2]:
from datasets import load_dataset

dataset_corpus = load_dataset("large_spanish_corpus", "ParaCrawl")

In [3]:
dataset_corpus

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 15510649
    })
})

In [4]:
subset = dataset_corpus['train'].select(range(1000000))

In [5]:
subset[50:60]

{'text': ['Su presencia en la tierra como poderes fantasma operando a través de sustitutos humanos serán expuestos para que todos los vean.',
  'Típica imagen de Hollywood de una invasión alienígena hostil por parte de naves aéreas extraterrestres de alta tecnología.',
  'o combinación de los anteriores?',
  'En la visión gnóstica, la divulgación o revelación no es el momento en que las Autoridades gubernamentales anuncian la presencia de extraterrestres en la tierra, haciéndolo así "oficial".',
  'Nada hecho oficial en este mundo cuenta para nada más que para otra táctica en el engaño globalista, avanzando en la agenda de las Autoridades.',
  'Sin embargo, para que los arcontes hagan su movimiento final, deberán revelarse.',
  'Debido a que su naturaleza es el engaño, lo harán de forma engañosa.',
  'Incluso al mostrar su presencia, jugando su carta de triunfo sobre la especie humana, ellos van a engañar, porque eso es todo lo que ellos hacen, todo lo que pueden hacer.',
  'La prueba 

In [6]:
subset.column_names

['text']

In [7]:
subset['text'][0:5]

['lavado de cerebro a través de los medios de comunicación, y amenaza de fuerza a través de los militares.',
 'Sin un constante aluvión de doble cañón, requiriendo la complicidad de los seres humanos para reprimir y engañar a sus semejantes, su tan cacareada magia rápidamente se desvanecería y se disiparía.',
 'En realidad, el Nuevo OM sólo se puede mantener la ilusión de supremacía mágica, siempre y cuando reprima y desvíe el potencial humano, donde mora la verdadera magia: es decir, en la capacidad innata de nuestra especie de magia interactiva con los poderes de animación de la diosa planetaria.',
 'A menos que el Nuevo OM pueda todo el tiempo suprimir la capacidad de una manera brutal, natural y espontáneamente se afirmará a sí misma.',
 'Cuando lo haga, la verdadera magia del Anthropos, el "niño luminoso," entrará de inmediato en acción.']

In [8]:
len(subset['text'][0])

104

In [9]:
from paths import EXTERNAL_DIR


In [ ]:
import os
import pickle
import nltk
from nltk.data import path as nltk_path
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords

# Ruta personalizada
custom_path = EXTERNAL_DIR
nltk_path.clear()
nltk_path.append(custom_path)

# Descargar recursos
nltk.download('punkt', download_dir=custom_path)
nltk.download('stopwords', download_dir=custom_path)

# Cargar tokenizer de oraciones desde el .pickle (no falla)
spanish_pickle_path = os.path.join(custom_path, "tokenizers", "punkt", "spanish.pickle")
with open(spanish_pickle_path, "rb") as f:
    sentence_tokenizer = pickle.load(f)

# Reemplazo de word_tokenize sin usar sent_tokenize()
word_tokenizer = TreebankWordTokenizer()

# Texto de prueba
text = "Hola, ¿cómo estás? Esto es una prueba de tokenización."

# Procesamiento
sentences = sentence_tokenizer.tokenize(text)
tokens = word_tokenizer.tokenize(text)  # SIN usar word_tokenize()
filtered = [t for t in tokens if t.lower() not in stopwords.words("spanish")]

# Resultado
print("Frases:", sentences)
print("Tokens:", tokens)
print("Tokens sin stopwords:", filtered)

Frases: ['Hola, ¿cómo estás?', 'Esto es una prueba de tokenización.']
Tokens: ['Hola', ',', '¿cómo', 'estás', '?', 'Esto', 'es', 'una', 'prueba', 'de', 'tokenización', '.']
Tokens sin stopwords: ['Hola', ',', '¿cómo', '?', 'prueba', 'tokenización', '.']


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jssdev/Dev/Learning/Platzi/platzi-
[nltk_data]     embeddings/data/external...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jssdev/Dev/Learning/Platzi/platzi-
[nltk_data]     embeddings/data/external...
[nltk_data]   Package stopwords is already up-to-date!


## Procesamiento de texto

In [20]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short
from nltk.data import path as nltk_path
import os

def clean_text(sentence_batch):
    # Ajusta path a recursos NLTK
    custom_path = EXTERNAL_DIR
    nltk_path.clear()
    nltk_path.append(custom_path)

    # Tokenizador sin dependencia de sent_tokenize
    tokenizer = TreebankWordTokenizer()
    stop_words = set(stopwords.words('spanish'))

    cleaned_text_list = []

    for text in sentence_batch['text']:
        # Minúsculas
        text = text.lower()

        # Eliminar URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)

        # Eliminar menciones y hashtags
        text = re.sub(r'\@\w+|\#\w+', '', text)

        # Eliminar puntuación, números, palabras cortas
        text = strip_punctuation(text)
        text = strip_numeric(text)
        text = strip_short(text, minsize=2)

        # Tokenizar y filtrar stopwords
        tokens = tokenizer.tokenize(text)
        filtered = [word for word in tokens if word not in stop_words]

        cleaned_text_list.append(filtered)

    return {'text': cleaned_text_list}


In [21]:
sentences_corpus = subset.map(clean_text, batched=True)

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [22]:
sentences_corpus['text'][:3]

[['lavado',
  'cerebro',
  'través',
  'medios',
  'comunicación',
  'amenaza',
  'fuerza',
  'través',
  'militares'],
 ['constante',
  'aluvión',
  'doble',
  'cañón',
  'requiriendo',
  'complicidad',
  'seres',
  'humanos',
  'reprimir',
  'engañar',
  'semejantes',
  'tan',
  'cacareada',
  'magia',
  'rápidamente',
  'desvanecería',
  'disiparía'],
 ['realidad',
  'nuevo',
  'om',
  'sólo',
  'puede',
  'mantener',
  'ilusión',
  'supremacía',
  'mágica',
  'siempre',
  'reprima',
  'desvíe',
  'potencial',
  'humano',
  'mora',
  'verdadera',
  'magia',
  'decir',
  'capacidad',
  'innata',
  'especie',
  'magia',
  'interactiva',
  'poderes',
  'animación',
  'diosa',
  'planetaria']]

## Carga y uso de modelo de embeddings Word2Vec

In [ ]:
model = Word2Vec()

# Podemos guardar el modelo para uso futuro
model.save("word2vec.model")

In [ ]:
##comida, ser, reina, television


## Almacenamiento de embeddings